# Run Ollama in Colab and use in local machine

Copy the code below to your Colab notebook.

> Running Colab in ssh is against the terms of service and risks getting your account kicked out. Please be careful and use it at your own risk.

In [ ]:
# Download and run the Ollama Linux install script
!curl https://ollama.ai/install.sh | sh
!command -v systemctl >/dev/null && sudo systemctl stop ollama

# Start Serve that runs Ollama
!pip install aiohttp pyngrok
!ngrok authtoken "your_ngrok_token"  # create an account at https://ngrok.com

import os
import asyncio
from aiohttp import ClientSession

# Set LD_LIBRARY_PATH so the system NVIDIA library becomes preferred
# over the built-in library. This is particularly important for
# Google Colab which installs older drivers
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

async def run(cmd):
  '''
  run is a helper function to run subcommands asynchronously.
  '''
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )


await asyncio.gather(
    run(['ollama', 'serve']),
    run(['ngrok', 'http', '--log', 'stderr', '11434']),
)

# Cheatsheet
Run the following command in your terminal.

```zsh
echo 'function collama() {
  export OLLAMA_HOST="$@"
}' >> ~/.zshrc

source ~/.zshrc
```

Then you can use this serve like a local ollama.
```zsh
collama "your_endpoint_url.ngrok-free.app"
ollama run llama2
```
